In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import sys
from PIL import Image, ImageFont, ImageDraw
font = ImageFont.truetype("arialbd.ttf", size=40)

left_fitx = [0,0,0]
right_fitx = [0,0,0]
ploty = 0
left_fit = 0
right_fit = 0
number_of_frames = 10
right_x_indeces_xx_frames = []
left_x_indeces_xx_frames = []

In [2]:
def cal_undistort(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [3]:
def corners_warp(img, nx, ny,action):
    offset = 100
    img_size = (img.shape[1], img.shape[0])

    #for full/half lanes
    src = np.float32([[278,704],[600,453],(727,453),[1111,704]])
    # for half lanes covering the full image
    dst = np.float32([[278,704],[278,0],[1111,0],[1111,704]])
    # for full lanes covering the full image
    if action is 'warp':
        M = cv2.getPerspectiveTransform(src, dst)
    elif action is 'unwarp':
        M = cv2.getPerspectiveTransform(dst, src)
    
    warped = cv2.warpPerspective(img, M, img_size)
    
    return warped, M

In [4]:
def fit_polynomial(leftx, lefty, rightx, righty, image):
  
    # Generate x and y values for plotting
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0] )

    if leftx.size==0:
        left_fitx = 0
        right_fitx = 0
        left_fit = 0
        right_fit = 0
        print('one of the arrays is ZERO!')
    # Fit new polynomials
    else:
            ### Fit a second order polynomial to each using `np.polyfit` ###
        left_fit = np.polyfit(lefty,leftx,2)
        right_fit = np.polyfit(righty,rightx,2)

        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    return left_fitx,right_fitx,ploty , left_fit,right_fit


In [5]:
def blind_lane_search(binary_warped):
   
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[np.int_(binary_warped.shape[0]//1.33):,:], axis=0)

    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])       # save the highest point location on left side for base
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint   # save the highest point location on left side for base

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 7
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero_coordinates = np.nonzero(binary_warped) #CHECK IF THIS IS OKAY
    nonzeroy = np.array(nonzero_coordinates[0])
    nonzerox = np.array(nonzero_coordinates[1])

    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current-margin  
        win_xleft_high = leftx_current+margin  
        win_xright_low = rightx_current-margin  
        win_xright_high = rightx_current+margin  

        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 

        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy <= win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox <= win_xleft_high)).nonzero()[0] # just taking x coordinates
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy <= win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox <= win_xright_high)).nonzero()[0] # just taking x coordinates

        # Append these X indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        ### recenter next window if more pixels found that the minimum threshold ###
        if len(good_left_inds>1) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds>1) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)


    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    
    left_fitx,right_fitx,ploty, left_fit, right_fit = fit_polynomial(leftx, lefty, rightx, righty, binary_warped)
    
    ## FOR PROCESSING IMAGES ONLY - BEGIN
    """
    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plot the left and right polynomials on the lane lines
    global fig_num
    fig.add_subplot(rows, columns, fig_num)
    fig_num+=1
    plt.title('Blind search for lanes')
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.imshow(out_img)"""
    ## FOR PROCESSING IMAGES ONLY - END
    
    return left_fitx, right_fitx, ploty, left_fit, right_fit

In [6]:
def focused_lane_search(binary_warped):# , left_fitx, right_fitx, ploty, left_fit, right_fit):
    margin = 100
        
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Fit new polynomials
    left_fitx_new,right_fitx_new,ploty_new , left_fit_new, right_fit_new = fit_polynomial(leftx, lefty, rightx, righty, binary_warped)
    
    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    
    ## FOR PROCESSING IMAGES ONLY - BEGIN
    """
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    # Generate a polygon to illustrate the search window area
    left_lane_window1 = np.transpose([left_fitx_new-margin,ploty_new])
    left_lane_window2 = np.flipud(np.transpose([left_fitx_new+margin,ploty_new]))
    left_lane_points = np.vstack((left_lane_window1,left_lane_window2))
  
    right_lane_window1 = np.transpose([right_fitx_new-margin,ploty_new])
    right_lane_window2 = np.flipud(np.transpose([right_fitx_new+margin,ploty_new]))
    right_lane_points = np.vstack((right_lane_window1,right_lane_window2))

    
    global fig_num
    fig.add_subplot(rows, columns, fig_num)
    fig_num+=1
    plt.title('Focused search for lanes')
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_lane_points]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_lane_points]), (0,255, 0))
    
    result_img = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    

    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.imshow(result_img)
    """
    ## FOR PROCESSING IMAGES ONLY - END
    

    return left_fitx_new, right_fitx_new, ploty_new, left_fit_new, right_fit_new

In [7]:
def measure_curvature_real(ploty, left_fitx, right_fitx):
    
    ## Calculates the curvature of polynomial functions in meters.
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty*ym_per_pix)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_radius = ((1 + (2*left_fit_cr[0]*y_eval + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_radius = ((1 + (2*right_fit_cr[0]*y_eval + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        
    avg_radius = ((left_radius+right_radius)/2).astype(int)
    radius_text_string = 'Radius of curvature = ' +str(avg_radius)+ '(m)'
    
    return radius_text_string

In [8]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    grad_binary = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    # Apply threshold
    return grad_binary

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    # Apply threshold
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    return mag_binary

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))*180/np.pi
    dir_binary =  np.zeros_like(absgraddir)
    # Apply threshold
    dir_binary[(absgraddir <= thresh[0])] = 1# & (absgraddir >= thresh[1])] = 1

    return dir_binary

In [9]:
def offset_calculator(left_fitx,right_fitx,x_axis):
    

    xm_per_pix = 3.7/(right_fitx[-1] - left_fitx[-1]) # meters per pixel in x dimension
    lane_center = (right_fitx[-1] + left_fitx[-1] )//2
    camera_center = x_axis//2
    offset = np.around(xm_per_pix*(camera_center - lane_center),2)
    if (offset > 0):
        offset_text = 'Vehicle is ' + str(np.absolute(offset)) + 'm left of Center'
    elif (offset<=0):
        offset_text = 'Vehicle is ' + str(np.absolute(offset)) + 'm right of Center'
    
    return offset_text

In [10]:
def process_images (image):
    
    global left_fitx, right_fitx, ploty, left_fit, right_fit, right_x_indeces_xx_frames, left_x_indeces_xx_frames
    nx = 9
    ny = 6
    imgpoints = np.load('camera_imgpoints.npy')
    objpoints = np.load('camera_objpoints.npy')
 
    ## UNDISTORT AND WARP THE ROAD IMAGES TO SEE THE LANES ##
    # CALIBRATE CAMERA
    undistorted = cal_undistort(image, objpoints, imgpoints)
    # Unwarp real images = make them straight
    warped_img,M = corners_warp(undistorted,nx,ny,'warp')
    
    # Choose a Sobel kernel size
    ksize = 9 # change it to 9 later # Choose a larger odd number to smooth gradient measurements

    # GRADIENT BASED LANE SEGREGATION
    gray = cv2.cvtColor(warped_img, cv2.COLOR_RGB2GRAY) #warped_img
    # Apply each of the thresholding functions    
    gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    mag_binary = mag_thresh(gray, sobel_kernel=ksize, mag_thresh=(30, 100))
    dir_binary = dir_threshold(gray, sobel_kernel=19, thresh=(20,135))

    # COLORSPACE BASED LANE SEGREGATION
   
    hls = cv2.cvtColor(warped_img, cv2.COLOR_RGB2HLS)
    hsv = cv2.cvtColor(warped_img, cv2.COLOR_RGB2HSV)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    
    S_thresh = (180, 255)
    S_binary = np.zeros_like(S)
    S_binary[(S > S_thresh[0]) & (S <= S_thresh[1])] = 1
    
    H_thresh = (15,60)#15,60 works
    H_binary = np.zeros_like(H)
    H_binary[(H > H_thresh[0]) & (H <= H_thresh[1])] = 1
    
    L_thresh = (180,255)#150-190 for yellow and 190-255 for white
    L_binary = np.zeros_like(L)
    L_binary[(L > L_thresh[0]) & (L <= L_thresh[1])] = 1
    
    combined_LorSorX = np.zeros_like(H_binary)
    combined_LorSorX[( ((L_binary == 1) | (S_binary == 1) | (gradx == 1)))] = 1
    
    ## FOR PROCESSING VIDEOS ONLY - BEGIN
    lane_difference = np.subtract(right_fitx,left_fitx)
    
    ## SANITY CHECK 1 - Left lane x indeces should never greater than right lane x indeces 
    ## and the gap at the bottom of the image between the two lanes should not be less than 400 pixels
    
    if (all(lDiff>0 for lDiff in lane_difference)) | (right_fitx[-1]-left_fitx[-1]>400) :
        wrong_lane_lines_detected = 0
    else:
        wrong_lane_lines_detected = 1
    
    ## If wrong lanes are detected, then restart with the blind lane search method
    ## , otherwise continue with focused lane search method
    
    if wrong_lane_lines_detected is 1: 
        left_fitx,right_fitx,ploty,left_fit, right_fit = blind_lane_search(combined_LorSorX)
        #print('blind')
        #right_x_indeces_xx_frames = []
        #left_x_indeces_xx_frames = []
    else:
        left_fitx,right_fitx,ploty,left_fit, right_fit = focused_lane_search(combined_LorSorX)
        #print('focused')
        
    ## SANITY CHECK 2 - if no points were identified in the binary thresholded images 
    ## then use the previous frame's lane markers
    if left_fitx is 0 or right_fitx is 0:
        right_fitx = right_x_indeces_xx_frames[-1]
        left_fitx = left_x_indeces_xx_frames[-1]
    
    ## If not enough frames present in the pipeline, then use the current x indeces
    ## otherwise use an average of previous and current frame indeces to display the lane lines
    
    right_x_indeces_xx_frames.append(right_fitx)
    if len(right_x_indeces_xx_frames)<number_of_frames:
        right_x = right_fitx
    else:
        right_x = np.array(right_x_indeces_xx_frames).mean(axis=0)  
        right_x_indeces_xx_frames.pop(0)
    
    left_x_indeces_xx_frames.append(left_fitx)    
    if len(left_x_indeces_xx_frames)<number_of_frames:
        left_x = left_fitx
    else:
        left_x = np.array(left_x_indeces_xx_frames).mean(axis=0)  ## fix this error
        left_x_indeces_xx_frames.pop(0)
    ## FOR PROCESSING VIDEOS ONLY - END
    
    
    ## FOR PRINTING IMAGES ONLY - BEGIN
    #left_fitx,right_fitx,ploty,left_fit, right_fit = blind_lane_search(combined_LorSorX)
    #right_x = right_fitx
    #left_x = left_fitx
    ## FOR PRINTING IMAGES ONLY - END
    
    right = np.transpose(np.array([right_x,ploty]))
    left = np.flipud(np.transpose(np.array([left_x,ploty])))
    
    
    poly_points = np.vstack((right,left))
    warp_zero = np.zeros_like(warped_img)
    lane_poly = cv2.fillPoly(warp_zero, np.int_([poly_points]), (0,255, 0)) 
    unwarped_lanes,M = corners_warp(lane_poly,nx,ny,'unwarp')
   
    final_img = cv2.addWeighted(undistorted,1,unwarped_lanes,.5,0)
    
    radius_text_string = measure_curvature_real(ploty, left_fitx, right_fitx)
    offset_string = offset_calculator (left_fitx, right_fitx,final_img.shape[1])
    
    final_img = cv2.putText(final_img,radius_text_string,(100,100),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255, 255, 255),2)#,lineType=cv2.LINE_AA)
    final_img = cv2.putText(final_img,offset_string,(100,180),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255, 255, 255),2)#,lineType=cv2.LINE_AA)
    
    #image_with_lanes = cv2.addWeighted(image_with_lanes,1,left_lane_poly,.8,0)
    
    #plt.imshow(L_binary)"""


    return(final_img)

In [12]:
#%matplotlib qt

"""### DEFAULT PROGRAM TO RUN IMAGES THROUGH

all_test_images = glob.glob('CarND-Advanced-Lane-Lines/test_images/*.jpg')

rows = len(all_test_images)
columns = 1

fig=plt.figure(figsize=(70, 10*rows))
fig_num=1

for index,file_name in enumerate(all_test_images):
    image = mpimg.imread(file_name)
    
     
    fig.add_subplot(rows, columns, fig_num)
    fig_num+=1
    plt.title('Original Image')
    plt.imshow(image)
    
    final_img1 = process_images(image)
    
    fig.add_subplot(rows, columns, fig_num)
    fig_num+=1
    plt.title('Final lane with poly fit')    

    

    plt.imshow(final_img1)#,cmap='gray')
    
    #fig.add_subplot(rows, columns, fig_num)
    #fig_num+=1
    #plt.title('HandSorX')
    #plt.imshow(final_img2,cmap='gray')

    """


"### DEFAULT PROGRAM TO RUN IMAGES THROUGH\n\nall_test_images = glob.glob('CarND-Advanced-Lane-Lines/test_images/*.jpg')\n\nrows = len(all_test_images)\ncolumns = 1\n\nfig=plt.figure(figsize=(70, 10*rows))\nfig_num=1\n\nfor index,file_name in enumerate(all_test_images):\n    image = mpimg.imread(file_name)\n    \n     \n    fig.add_subplot(rows, columns, fig_num)\n    fig_num+=1\n    plt.title('Original Image')\n    plt.imshow(image)\n    \n    final_img1 = process_images(image)\n    \n    fig.add_subplot(rows, columns, fig_num)\n    fig_num+=1\n    plt.title('Final lane with poly fit')    \n\n    \n\n    plt.imshow(final_img1)#,cmap='gray')\n    \n    #fig.add_subplot(rows, columns, fig_num)\n    #fig_num+=1\n    #plt.title('HandSorX')\n    #plt.imshow(final_img2,cmap='gray')\n\n    "

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'CarND-Advanced-Lane-Lines/test_videos_output/challenge_video_sanyam_try.mp4' #project_video_output #solidYellowLeft

clip1 = VideoFileClip("CarND-Advanced-Lane-Lines/challenge_video.mp4")#.subclip(1,6)#project_video

white_clip = clip1.fl_image(process_images)
#white_clip = clip1.fl_image(lambda image: process_images(image,left_fit,right_fit)) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video CarND-Advanced-Lane-Lines/test_videos_output/challenge_video_sanyam_try.mp4
[MoviePy] Writing video CarND-Advanced-Lane-Lines/test_videos_output/challenge_video_sanyam_try.mp4


 34%|███████████████████████████▏                                                    | 165/485 [03:36<07:12,  1.35s/it]